# Basic neural bag-of-words text classifier with Thinc

This notebook shows how to implement a simple neural text classification model in Thinc.

In [1]:
!pip install "thinc>=8.0.0a0" ml_datasets "tqdm>=4.41" syntok

For simple and standalone tokenization, we'll use the [`syntok`](https://github.com/fnl/syntok) package and the following function:

In [2]:
from syntok.tokenizer import Tokenizer

def tokenize_texts(texts):
    tok = Tokenizer()
    return [[token.value for token in tok.tokenize(text)] for text in texts]

## Setting up the data

The `load_data` function loads the DBPedia Ontology dataset, converts and tokenizes the data and generates a simple vocabulary mapping. Instead of `ml_datasets.dbpedia` you can also try `ml_datasets.imdb` for the IMDB review dataset.

In [3]:
import ml_datasets
import numpy

def load_data():
    train_data, dev_data = ml_datasets.dbpedia(limit=2000)
    train_texts, train_cats = zip(*train_data)
    dev_texts, dev_cats = zip(*dev_data)
    unique_cats = list(numpy.unique(numpy.concatenate((train_cats, dev_cats))))
    nr_class = len(unique_cats)
    print(f"{len(train_data)} training / {len(dev_data)} dev\n{nr_class} classes")

    train_y = numpy.zeros((len(train_cats), nr_class), dtype="f")
    for i, cat in enumerate(train_cats):
        train_y[i][unique_cats.index(cat)] = 1
    dev_y = numpy.zeros((len(dev_cats), nr_class), dtype="f")
    for i, cat in enumerate(dev_cats):
        dev_y[i][unique_cats.index(cat)] = 1

    train_tokenized = tokenize_texts(train_texts)
    dev_tokenized = tokenize_texts(dev_texts)
    # Generate simple vocab mapping, <unk> is 0
    vocab = {}
    count_id = 1
    for text in train_tokenized:
        for token in text:
            if token not in vocab:
                vocab[token] = count_id
                count_id += 1
    # Map texts using vocab
    train_X = []
    for text in train_tokenized:
        train_X.append(numpy.array([vocab.get(t, 0) for t in text]))
    dev_X = []
    for text in dev_tokenized:
        dev_X.append(numpy.array([vocab.get(t, 0) for t in text]))
    return (train_X, train_y), (dev_X, dev_y), vocab, train_texts, dev_texts

(train_X, train_y), (dev_X, dev_y), vocab, train_texts, dev_texts = load_data()

2000 training / 2000 dev
14 classes


In [4]:
print(train_X[1].shape)
print(train_y[1])

(38,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


## Defining the model and config

The model takes a list of 2-dimensional arrays (the tokenized texts mapped to vocab IDs) and outputs a 2d array. Because the embed layer's `nV` dimension (the number of entries in the lookup table) depends on the vocab and the training data, it's passed in as an argument and registered as a **reference**. This makes it easy to retrieve it later on by calling `model.get_ref("embed")`, so we can set its `nV` dimension.

In [8]:
from typing import List
import thinc
from thinc.api import Model, list2ragged, list2array, chain, with_array, reduce_mean, Softmax
from thinc.types import Array2d

@thinc.registry.layers("EmbedPoolTextcat.v1")
def EmbedPoolTextcat(embed: Model[Array2d, Array2d]) -> Model[List[Array2d], Array2d]:
    with Model.define_operators({">>": chain}):
        # Original line:
        # model = list2ragged() >> with_array(embed) >> reduce_mean() >> Softmax()
        # My change:
        model = with_array(embed) >> list2ragged() >> reduce_mean() >> Softmax()
    model.set_ref("embed", embed)
    return model
    


CONFIG = """
[hyper_params]
width = 64

[model]
@layers = "EmbedPoolTextcat.v1"

[model.embed]
@layers = "Embed.v1"
nO = ${hyper_params:width}

[optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.001

[training]
batch_size = 8
n_iter = 10
"""
from thinc.api import registry, Config

C = registry.make_from_config(Config().from_str(CONFIG))

batch_size = C["training"]["batch_size"]
optimizer = C["optimizer"]
model = C["model"]
model.get_ref("embed").set_dim("nV", len(vocab) + 1)



In [9]:
model.initialize(X=train_X[:5], Y=train_y[:5])

Alternativelly, the model could be defined withouth operator overloading as:

```python
CONFIG = """
[hyper_params]
width = 64

[optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.001

[training]
batch_size = 8
n_iter = 10
"""

from thinc.api import registry, Config
C = registry.make_from_config(Config().from_str(CONFIG))

batch_size = C["training"]["batch_size"]
optimizer = C["optimizer"]
width = C["hyper_params"]["width"]

model = chain(
    with_array(Embed(nO=width, nV=len(vocab) + 1)),
    list2ragged(),
    reduce_mean(),
    Softmax(nO=nr_classes)
)

model.initialize(X=train_X[:5], Y=train_y[:5])
```

In [10]:
def evaluate_model(model, dev_X, dev_Y, batch_size):
    correct = 0.0
    total = 0.0
    for X, Y in model.ops.multibatch(batch_size, dev_X, dev_Y):
        Yh = model.predict(X)
        for j in range(len(Yh)):
            correct += Yh[j].argmax(axis=0) == Y[j].argmax(axis=0)
        total += len(Y)
    return float(correct / total)

---

## Training the model

In [11]:
from thinc.api import fix_random_seed
from tqdm.notebook import tqdm

fix_random_seed(0)
for n in range(C["training"]["n_iter"]):
    loss = 0.0
    batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
    for X, Y in tqdm(batches, leave=False):
        Yh, backprop = model.begin_update(X)
        d_loss = []
        for i in range(len(Yh)):
            d_loss.append(Yh[i] - Y[i])
            loss += ((Yh[i] - Y[i]) ** 2).sum()
        backprop(numpy.array(d_loss))
        model.finish_update(optimizer)
    score = evaluate_model(model, dev_X, dev_y, batch_size)
    print(f"{n}\t{loss:.2f}\t{score:.3f}")

0	1838.46	0.496


1	1616.91	0.635


2	1182.37	0.768


3	781.83	0.866


4	472.20	0.895


5	271.41	0.908


6	156.96	0.915


7	92.86	0.915


8	55.64	0.919


9	34.02	0.923
